In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [0]:
pip install livelossplot

# COVID-19 Minor Project

In [0]:
import gc
import cv2
import glob
import keras
import numpy as np 
import pandas as pd 
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from livelossplot.tf_keras import PlotLossesCallback

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionResNetV2, ResNet152V2, DenseNet169

import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential

from random import choice, uniform, random, randint
from operator import itemgetter, attrgetter

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


##Data Pre-processing

In [0]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
def load_images_from_folder(folder):
    cnt=0
    for filename in os.listdir(folder):
        cnt = cnt+1
    return cnt

In [0]:
cvd = load_images_from_folder('/content/drive/My Drive/data')
cvd

2492

In [0]:
def prep_img(trg, dataset, size):
  print("...Preparing images..")
  X_train = np.zeros((size, 224, 224, 3))
  count = 0
  for fig in os.listdir(dataset):
          #load images into images of size 224x224x3
          imag = cv2.imread("/content/drive/My Drive/data"+"/"+fig)
          imag = cv2.resize(imag, (224, 224))
          #img = image.load_img("/content/drive/My Drive/data"+"/"+fig, target_size=(224, 224))
          #x = image.img_to_array(imag)
          imag = imag/255
          #x = preprocess_input(x)
          #x = x/255
          #print(fig)

          X_train[count] = imag
          print("Processing image: ", count+1, ", ", fig)
          count += 1
  print("...Done...")
    
  return X_train

In [0]:
trg_tot = '/content/drive/My Drive/'
X_c = prep_img(trg_tot, 'data', cvd)

...Preparing images..
Processing image:  1 ,  Non-Covid (545).png
Processing image:  2 ,  Non-Covid (628).png
Processing image:  3 ,  Non-Covid (493).png
Processing image:  4 ,  Non-Covid (563).png
Processing image:  5 ,  Non-Covid (652).png
Processing image:  6 ,  Non-Covid (609).png
Processing image:  7 ,  Non-Covid (530).png
Processing image:  8 ,  Non-Covid (561).png
Processing image:  9 ,  Non-Covid (526).png
Processing image:  10 ,  Non-Covid (529).png
Processing image:  11 ,  Non-Covid (611).png
Processing image:  12 ,  Non-Covid (58).png
Processing image:  13 ,  Non-Covid (64).png
Processing image:  14 ,  Non-Covid (603).png
Processing image:  15 ,  Non-Covid (588).png
Processing image:  16 ,  Non-Covid (637).png
Processing image:  17 ,  Non-Covid (533).png
Processing image:  18 ,  Non-Covid (565).png
Processing image:  19 ,  Non-Covid (621).png
Processing image:  20 ,  Non-Covid (496).png
Processing image:  21 ,  Non-Covid (654).png
Processing image:  22 ,  Non-Covid (555).png

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-ff1d33ffd4d2>", line 2, in <module>
    X_c = prep_img(trg_tot, 'data', cvd)
  File "<ipython-input-8-b58be9030b65>", line 7, in prep_img
    imag = cv2.imread("/content/drive/My Drive/data"+"/"+fig)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinner

KeyboardInterrupt: ignored

In [0]:
def create_labels(dataset):
  labels=[]
  for fig in os.listdir(dataset):
    if fig[0]=='C':
      labels.append('Covid')
    elif fig[0]=='N':
      labels.append('Normal')
  return labels
lbl = create_labels('data')

In [0]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded.shape)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #print(onehot_encoded)

    y = onehot_encoded
    #print(y)
    return y, label_encoder, onehot_encoder
y, label_encoder, onehot_encoder = prepare_labels(lbl)
y.shape

In [0]:
X_trainn, X_test, y_trainn, y_test = train_test_split(X_c, y, test_size=0.25, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainn, y_trainn, test_size=0.2, random_state=42, stratify=y_trainn)

In [0]:
cnt = cnt_1 = cnt_t = cnt_t_1 = a=b=0
for lb in y_train:
  if lb[0] == 0:
    cnt_1 = cnt_1 + 1
  elif lb[0] == 1:
    cnt = cnt +1
for lb in y_test:
  if lb[0] == 0:
    cnt_t_1 = cnt_t_1 + 1
  elif lb[0] == 1:
    cnt_t = cnt_t +1
for lb in y_val:
  if lb[0] == 0:
    a = a + 1
  elif lb[0] == 1:
    b = b +1
print (cnt,cnt_1,cnt_t,cnt_t_1,a,b)

## Data Augmentation

In [0]:
trainAug = ImageDataGenerator(
	rotation_range=15,
  shear_range=0.2,
  horizontal_flip=True,
	fill_mode="nearest")

## VGG-16

In [0]:
pretrained_model = VGG16(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )
for layer in pretrained_model.layers:
    layer.trainable = False

model = Sequential()
    # first (and only) set of FC => RELU layers
model.add(AveragePooling2D((2, 2), name='avg_pool'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
    # softmax classifier
model.add(Dense(2,activation='sigmoid'))



pretrainedInput = pretrained_model.input
pretrainedOutput = pretrained_model.output
output = model(pretrainedOutput)
model = Model(pretrainedInput, output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001, mode='auto')
checkpoint = ModelCheckpoint("model_vgg16_weights.h5", monitor='val_accuracy',save_weights_only=True, mode='max', verbose=1)
callbacks = [PlotLossesCallback(), checkpoint, reduce_lr]

In [0]:
H = model.fit_generator(trainAug.flow(X_train, y_train, batch_size=64), steps_per_epoch=len(X_train)//64, validation_data=(X_val, y_val), validation_steps=len(X_val)//64, epochs=1000, callbacks=callbacks, use_multiprocessing=True)
#H = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

In [0]:
# summarize history for accuracy
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

### Prediction and Prepping

In [0]:
inte1 = onehot_encoder.inverse_transform(y_test)
fi1 = label_encoder.inverse_transform(inte1)
Y_pred = model.predict(X_test)
inte = onehot_encoder.inverse_transform(Y_pred)
fi = label_encoder.inverse_transform(inte)

In [0]:
print("Precision Score : ",precision_score(fi1, fi, pos_label='Covid', average='binary'))
print("Recall Score : ",recall_score(fi1, fi, pos_label='Covid',average='binary'))
print('Accuracy Score : ' + str(accuracy_score(fi1,fi)))
print('F1 Score : ' + str(f1_score(fi1,fi, pos_label='Covid',average='binary')))
cm1=confusion_matrix(fi1, fi)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

### Confusion Matrix

In [0]:
print('Confusion Matrix')
print(confusion_matrix(fi1, fi))
print('Classification Report')
target_names = ['Covid', 'Normal']
print(classification_report(fi1, fi, target_names=target_names))

In [0]:
import seaborn as sns
con_mat = confusion_matrix(inte1,inte)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
 
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = target_names, 
                     columns = target_names)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

### Area under Curve

In [0]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names): # all_labels: no of the labels
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        plt.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    plt.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)
multiclass_roc_auc_score(fi1, fi)

In [0]:
fpr, tpr, threshold = roc_curve(inte1, inte)
roc_auc = auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## InceptionResNetV2

In [0]:
pretrained_model1 = InceptionResNetV2(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )
for layer in pretrained_model1.layers:
    layer.trainable = False

model1 = Sequential()
    # first (and only) set of FC => RELU layers
model1.add(AveragePooling2D((2, 2), name='avg_pool'))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))

model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.3))
    # softmax classifier
model1.add(Dense(2,activation='sigmoid'))



pretrainedInput1 = pretrained_model1.input
pretrainedOutput1 = pretrained_model1.output
output1 = model1(pretrainedOutput1)
model1 = Model(pretrainedInput1, output1)
model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model1.summary()

In [0]:
H1 = model1.fit_generator(trainAug.flow(X_train, y_train, batch_size=64), steps_per_epoch=len(X_train)//64, validation_data=(X_val, y_val), validation_steps=len(X_val)//64, epochs=500)
#H = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

In [0]:
# summarize history for accuracy
plt.plot(H1.history['accuracy'])
plt.plot(H1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(H1.history['loss'])
plt.plot(H1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [0]:
inte11 = onehot_encoder.inverse_transform(y_test)
fi11 = label_encoder.inverse_transform(inte11)
Y_pred1 = model1.predict(X_test)
inte1 = onehot_encoder.inverse_transform(Y_pred1)
fi1 = label_encoder.inverse_transform(inte1)

In [0]:
print("Precision Score : ",precision_score(fi11, fi1, pos_label='Covid', average='binary'))
print("Recall Score : ",recall_score(fi11, fi1, pos_label='Covid',average='binary'))
print('Accuracy Score : ' + str(accuracy_score(fi11,fi1)))
print('F1 Score : ' + str(f1_score(fi11,fi1, pos_label='Covid',average='binary')))
cm1=confusion_matrix(fi11, fi1)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

In [0]:
print('Confusion Matrix')
print(confusion_matrix(fi11, fi1))
print('Classification Report')
target_names = ['Covid', 'Normal']
print(classification_report(fi11, fi1, target_names=target_names))

In [0]:
import seaborn as sns
con_mat = confusion_matrix(inte11,inte1)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
 
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = target_names, 
                     columns = target_names)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [0]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names): # all_labels: no of the labels
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        plt.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    plt.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)
multiclass_roc_auc_score(fi11, fi1)

In [0]:
fpr, tpr, threshold = roc_curve(inte11, inte1)
roc_auc = auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


## ResNet152V2

In [0]:
pretrained_model2 = ResNet152V2(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )
for layer in pretrained_model2.layers:
    layer.trainable = False

model2 = Sequential()
    # first (and only) set of FC => RELU layers
model2.add(AveragePooling2D((2, 2), name='avg_pool'))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
    # softmax classifier
model2.add(Dense(2,activation='sigmoid'))



pretrainedInput2 = pretrained_model2.input
pretrainedOutput2 = pretrained_model2.output
output2 = model2(pretrainedOutput2)
model2 = Model(pretrainedInput2, output2)
model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model2.summary()

In [0]:
H2 = model2.fit_generator(trainAug.flow(X_train, y_train, batch_size=64), steps_per_epoch=len(X_train)//64, validation_data=(X_val, y_val), validation_steps=len(X_val)//64, epochs=500)
#H = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

In [0]:
# summarize history for accuracy
plt.plot(H2.history['accuracy'])
plt.plot(H2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(H2.history['loss'])
plt.plot(H2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [0]:
inte12 = onehot_encoder.inverse_transform(y_test)
fi12 = label_encoder.inverse_transform(inte12)
Y_pred2 = model2.predict(X_test)
inte2 = onehot_encoder.inverse_transform(Y_pred2)
fi2 = label_encoder.inverse_transform(inte2)

In [0]:
print("Precision Score : ",precision_score(fi12, fi2, pos_label='Covid', average='binary'))
print("Recall Score : ",recall_score(fi12, fi2, pos_label='Covid',average='binary'))
print('Accuracy Score : ' + str(accuracy_score(fi12, fi2)))
print('F1 Score : ' + str(f1_score(fi12, fi2, pos_label='Covid',average='binary')))
cm1=confusion_matrix(fi12, fi2)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

In [0]:
print('Confusion Matrix')
print(confusion_matrix(fi12, fi2))
print('Classification Report')
target_names = ['Covid', 'Normal']
print(classification_report(fi12, fi2, target_names=target_names))

In [0]:
import seaborn as sns
con_mat = confusion_matrix(inte12,inte2)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
 
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = target_names, 
                     columns = target_names)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [0]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names): # all_labels: no of the labels
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        plt.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    plt.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)
multiclass_roc_auc_score(fi12, fi2)

In [0]:
fpr, tpr, threshold = roc_curve(inte12, inte2)
roc_auc = auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


## DenseNet201

In [0]:
pretrained_model3 = DenseNet169(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )
for layer in pretrained_model3.layers:
    layer.trainable = False

model3 = Sequential()
    # first (and only) set of FC => RELU layers
model3.add(AveragePooling2D((2, 2), name='avg_pool'))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.2))

model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.3))
    # softmax classifier
model3.add(Dense(2,activation='sigmoid'))



pretrainedInput3 = pretrained_model3.input
pretrainedOutput3 = pretrained_model3.output
output3 = model3(pretrainedOutput3)
model3 = Model(pretrainedInput3, output3)
model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model3.summary()

In [0]:
H3 = model3.fit_generator(trainAug.flow(X_train, y_train, batch_size=64), steps_per_epoch=len(X_train)//64, validation_data=(X_val, y_val), validation_steps=len(X_val)//64, epochs=500)
#H = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

In [0]:
# summarize history for accuracy
plt.plot(H3.history['accuracy'])
plt.plot(H3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(H3.history['loss'])
plt.plot(H3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [0]:
inte13 = onehot_encoder.inverse_transform(y_test)
fi13 = label_encoder.inverse_transform(inte13)
Y_pred3 = model3.predict(X_test)
inte3 = onehot_encoder.inverse_transform(Y_pred3)
fi3 = label_encoder.inverse_transform(inte3)

In [0]:
print("Precision Score : ",precision_score(fi13, fi3, pos_label='Covid', average='binary'))
print("Recall Score : ",recall_score(fi13, fi3, pos_label='Covid',average='binary'))
print('Accuracy Score : ' + str(accuracy_score(fi13, fi3)))
print('F1 Score : ' + str(f1_score(fi13, fi3, pos_label='Covid',average='binary')))
cm1=confusion_matrix(fi13, fi3)
sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )
specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

In [0]:
print('Confusion Matrix')
print(confusion_matrix(fi13, fi3))
print('Classification Report')
target_names = ['Covid', 'Normal']
print(classification_report(fi13, fi3, target_names=target_names))

In [0]:
import seaborn as sns
con_mat = confusion_matrix(inte13,inte3)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
 
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = target_names, 
                     columns = target_names)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [0]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names): # all_labels: no of the labels
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        plt.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    plt.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)
multiclass_roc_auc_score(fi13, fi3)

In [0]:
fpr, tpr, threshold = roc_curve(inte13, inte3)
roc_auc = auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()